In [1]:
import argparse
import yaml
import os, sys
import cv2
import time
import logging, logging.config

from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.visualizer import ColorMode

In [2]:
sys.path.insert(0, "..")

from helpers.detectron2 import LossEvalHook, CocoTrainer
from helpers.detectron2 import dt2predictions_to_list

# Prepare_data.py
Making the labels right

In [2]:
import geopandas as gpd

In [15]:
ROADS="/mnt/data-01/gsalamin/proj-roadsurf-b/02_Data/processed/shapefiles_gpkg/roads_polygons.shp"
TILES="/mnt/data-01/gsalamin/proj-roadsurf-b/02_Data/processed/json/tiles_aoi_z17.geojson"

OUTPUT_DIR="/mnt/data-01/gsalamin/proj-roadsurf-b/02_Data/processed/json"

written_files=[]


In [6]:
roads=gpd.read_file(ROADS)
tiles=gpd.read_file(TILES)

/home/gsalamin/objdet/lib/python3.8/site-packages/geopandas/_vectorized.py:143: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out


In [22]:
labels_gdf = roads.rename(columns={'BELAGSART': 'CATEGORY', 'road_width':'WIDTH'}).drop(columns=[
                    'DATUM_AEND', 'DATUM_ERST', 'ERSTELLUNG', 'ERSTELLU_1',
                    'REVISION_J', 'REVISION_M', 'GRUND_AEND', 'HERKUNFT', 'HERKUNFT_J',
                    'HERKUNFT_M', 'REVISION_Q', 'WANDERWEGE', 'VERKEHRSBE', 
                    'BEFAHRBARK', 'EROEFFNUNG', 'STUFE', 'RICHTUNGSG', 
                    'KREISEL', 'EIGENTUEME', 'VERKEHRS_1', 'NAME',
                    'TLM_STRASS', 'STRASSENNA', 'SHAPE_Leng',])
labels_gdf['SUPERCATEGORY']='road'
labels_gdf = labels_gdf.to_crs(epsg=4326)


In [23]:
labels_gdf.columns

Index(['OBJECTID', 'OBJEKTART', 'KUNSTBAUTE', 'CATEGORY', 'WIDTH', 'geometry',
       'SUPERCATEGORY'],
      dtype='object')

In [24]:
try:
    assert( labels_gdf.crs == tiles.crs ), f"CRS mismatching: labels' CRS = {labels_gdf.crs} != OK_tiles' CRS = {tiles.crs}"
except Exception as e:
    print(e)
    sys.exit(1)

In [25]:
GT_labels_gdf = gpd.sjoin(labels_gdf, tiles, how='inner', op='intersects')

In [26]:

# the following two lines make sure that no object is counted more than once in case it intersects multiple tiles
GT_labels_gdf = GT_labels_gdf[labels_gdf.columns]
GT_labels_gdf.drop_duplicates(inplace=True)
OTH_labels_gdf = labels_gdf[ ~labels_gdf.index.isin(GT_labels_gdf.index)]

# In the current case, OTH_labels_gdf should be empty

/home/gsalamin/objdet/lib/python3.8/site-packages/geopandas/_vectorized.py:175: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out


In [27]:
try:
    assert( len(labels_gdf) == len(GT_labels_gdf) + len(OTH_labels_gdf) ),\
        f"Something went wrong when splitting labels into Ground Truth Labels and Other Labels. Total no. of labels = {len(labels_gdf)}; no. of Ground Truth Labels = {len(GT_labels_gdf)}; no. of Other Labels = {len(OTH_labels_gdf)}"
except Exception as e:
    print(e)
    sys.exit(1)


In [28]:
GT_LABELS_GEOJSON = os.path.join(OUTPUT_DIR, f'ground_truth_labels.geojson')
OTH_LABELS_GEOJSON = os.path.join(OUTPUT_DIR, f'other_labels.geojson')

GT_labels_gdf.to_crs(epsg=4326).to_file(GT_LABELS_GEOJSON, driver='GeoJSON')
written_files.append(GT_LABELS_GEOJSON)

if not OTH_labels_gdf.empty:
    OTH_labels_gdf.to_crs(epsg=4326).to_file(OTH_LABELS_GEOJSON, driver='GeoJSON')
    written_files.append(OTH_LABELS_GEOJSON)

/home/gsalamin/objdet/lib/python3.8/site-packages/geopandas/io/file.py:234: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


# train_model.py

Definition of the constant

In [29]:


MODEL_ZOO_CHECKPOINT_URL = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"

MODEL_PTH_FILE = None
    
COCO_TRN_FILE = "COCO_trn.json"
COCO_VAL_FILE = "COCO_val.json"
COCO_TST_FILE = "COCO_tst.json"
        
DETECTRON2_CFG_FILE = '/home/gsalamin/Documents/GitHub/object-detector/examples/swimming-pool-detection/GE/detectron2_config_GE.yaml'

WORKING_DIR = "/mnt/data-01/gsalamin/proj-roadsurf-b/02_Data/processed/obj_detector"
SAMPLE_TAGGED_IMG_SUBDIR = "sample_training_images"
LOG_SUBDIR = "logs"
OUTPUT_DIR=WORKING_DIR+'/tests'   

os.chdir(WORKING_DIR)
# let's make the output directories in case they don't exist
for DIR in [SAMPLE_TAGGED_IMG_SUBDIR, LOG_SUBDIR]:
    if not os.path.exists(DIR):
        os.makedirs(DIR)

written_files = []

In [4]:
# ---- register datasets
# register_coco_instances("trn_dataset", {}, COCO_TRN_FILE, "")
# register_coco_instances("val_dataset", {}, COCO_VAL_FILE, "")
# register_coco_instances("tst_dataset", {}, COCO_TST_FILE, "")
register_coco_instances("oth_dataset", {}, "COCO_oth.json", "")

# registered_datasets = ['trn_dataset', 'val_dataset', 'tst_dataset', 'oth_dataset']
registered_datasets = ['oth_dataset']

    
registered_datasets_prefixes = [x.split('_')[0] for x in registered_datasets]



In [11]:
for dataset in registered_datasets:
    
    for d in DatasetCatalog.get(dataset)[0:min(len(DatasetCatalog.get(dataset)), 4)]:
        output_filename = "tagged_" + d["file_name"].split('/')[-1]
        output_filename = output_filename.replace('tif', 'png')
        
        img = cv2.imread(d["file_name"])  
        
        visualizer = Visualizer(img[:, :, ::-1], metadata=MetadataCatalog.get(dataset), scale=1.0)
        
        vis = visualizer.draw_dataset_dict(d)
        cv2.imwrite(os.path.join(SAMPLE_TAGGED_IMG_SUBDIR, output_filename), vis.get_image()[:, :, ::-1])
        written_files.append( os.path.join(WORKING_DIR, os.path.join(SAMPLE_TAGGED_IMG_SUBDIR, output_filename)) )
            


[11/02 13:08:43 d2.data.datasets.coco]: Loaded 38077 images in COCO format from COCO_oth.json
[11/02 13:08:43 d2.data.datasets.coco]: Loaded 38077 images in COCO format from COCO_oth.json


In [18]:
# cf. https://detectron2.readthedocs.io/modules/config.html#config-references
cfg = get_cfg()
cfg.merge_from_file(DETECTRON2_CFG_FILE)
cfg.OUTPUT_DIR = LOG_SUBDIR

In [19]:
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(MODEL_ZOO_CHECKPOINT_URL)

In [21]:
trainer = CocoTrainer(cfg)

[11/02 15:19:13 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

KeyError: "Dataset 'trn_dataset' is not registered! Available datasets are: coco_2014_train, coco_2014_val, coco_2014_minival, coco_2014_minival_100, coco_2014_valminusminival, coco_2017_train, coco_2017_val, coco_2017_test, coco_2017_test-dev, coco_2017_val_100, keypoints_coco_2014_train, keypoints_coco_2014_val, keypoints_coco_2014_minival, keypoints_coco_2014_valminusminival, keypoints_coco_2014_minival_100, keypoints_coco_2017_train, keypoints_coco_2017_val, keypoints_coco_2017_val_100, coco_2017_train_panoptic_separated, coco_2017_train_panoptic_stuffonly, coco_2017_train_panoptic, coco_2017_val_panoptic_separated, coco_2017_val_panoptic_stuffonly, coco_2017_val_panoptic, coco_2017_val_100_panoptic_separated, coco_2017_val_100_panoptic_stuffonly, coco_2017_val_100_panoptic, lvis_v1_train, lvis_v1_val, lvis_v1_test_dev, lvis_v1_test_challenge, lvis_v0.5_train, lvis_v0.5_val, lvis_v0.5_val_rand_100, lvis_v0.5_test, lvis_v0.5_train_cocofied, lvis_v0.5_val_cocofied, cityscapes_fine_in